# U1 Actividad NLP

En esta actividad se ha analizado el conjunto de datos de [OpinRankDatasetWithJudgments](http://kavita-ganesan.com/entity-ranking-data/) para hallar las respuestas a las preguntas que propone el enunciado.

Primero, nos descargamos el dataset:

In [ ]:
# If you haven't downloaded the dataset, then uncomment and run. Note unzipping is os dependent
# !curl https://github.com/kavgan/OpinRank/raw/master/OpinRankDatasetWithJudgments.zip -o "data/OpinRankDatasetWithJudgments.zip"
# !7z x -y "data/OpinRankDatasetWithJudgments.zip" -odata # 7zip / windows
# !unzip "data/OpinRankDatasetWithJudgments.zip" # mac

## Lectura de datos

Dentro del zip que hemos descargados encontramos la siguiente estructura de ficheros:

- data (dir)
  - cars (dir)
  - hotels (dir)
    - data (dir)
      - city-name (dir)
        - hotel-name (file)
          - *reviews here*
      - city-name.csv (file)
        - *hotels information*
    - judgments
      - city-name

Ignoramos los datos de coches, directorio `cars`, y nos centraremos en el directorio `hotels`. Dentro de `hotels` encontramos dos directorios, `data` y `judgments`. 

En el primer directorio, data, encontramos los datos sin procesar. Estos datos se agrupan por ciudades. Cada ciudad tiene un fichero `.csv` y un directorio con su nombre. En el csv encontramos la lista de hoteles de esa ciudad y datos asociados a estos. Entre estos datos destaca el atributo `doc_id`, que contiene el nombre del fichero donde se encuentran las review de los usuarios. El fichero de las review se encuentra dentro del directorio con el nombre de la ciudad a la que pertenece.

El `judgments` encontramos un directorio por cada ciudad. Dentro de cada directorio encontramos los resultados de las puntuaciones de relevancia.

Para información más detallada podéis consultar la documentación del dataset, `OpinRankDatasetWithJudgments.pdf`.

In [ ]:
import os
import pandas as pd

hotels = pd.DataFrame()

# Podemos iterar y unir todos los csv porque sabemos que solo hay 10 ciudades.
# Alternativamente, usariamos una función para cargar cada ciudad de forma independiente 
# como se ha hecho con las review de cada hotel.
for name in os.listdir("data/hotels/data"):
    if name.endswith(".csv"):
      hotelTmp = pd.read_csv(f"data/hotels/data/{name}", delimiter=',', index_col=False)
      hotels = pd.concat([hotels, hotelTmp], axis=0)

# Guardamos todos los nombres de las ciudades
cities = hotels.city.unique()

hotels.head()

In [ ]:
# Dada una fila/row del dataframe hotels del apartado anterior, nos devuelve un dataframe con las reviews
def getReviews(hotel):
  docId = hotel["doc_id"]
  city = hotel["city"]
  fileName = f"data/hotels/data/{city}/{docId}"
  try:
    reviews = pd.read_csv(fileName, delimiter="\t", index_col=False, header=None, encoding="iso-8859-1")
    reviews.columns = ["date", "title", "review", "empty"]
    reviews = reviews.drop(columns="empty") # last column is generated by a tab before end of line
    return reviews
  except FileNotFoundError:
    return pd.DataFrame()
  except Exception as e:
    if hasattr(e, 'message'):
      print(f"No se ha podido leer las reviews de {fileName}. Error {e.message}")
    else:
      print(e)
    return pd.DataFrame()

# test the function
reviews = getReviews(hotels.iloc[10])
reviews.head()

En el proceso de carga de las reviews vimos que los ficheros que contienen las review:
- Tienen una review por fila.
- El separador es un tabulador.
- El encode del fichero es `iso-8859-1`. Se puede abrir en `utf-8` y es legible excepto acentos y algunos caracteres especiales.

En el ejemplo podemos comprobar que la columna `date` no siempre va a tener valor.

## 1. ¿Qué partes de una habitación son las más mencionadas?

In this section, I'm going to use wordnet of each room to look for matches. First, I'm going to look for the definition of room that we need. Then use it's hyponyms to search in the reviews.

In [ ]:
# load dependencies
import re
import tqdm
# import nltk
# nltk.download('averaged_perceptron_tagger')
# nltk.download('universal_tagset')
# nltk.download("punkt")
from nltk import word_tokenize
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

lemmatizer = WordNetLemmatizer()

In [ ]:
# check the room definition that we need
for sense in wordnet.synsets('room'):
    print(sense)
    print(sense.definition())
    print(sense.examples())
    print("-"*10)

# Synset('room.n.01') is the definition that I'm looking for

In [ ]:
room = wordnet.synset('room.n.01')
room.hyponyms()

# now we need to "correct" the spelling of user words. Ex: some reviews wrote "bathrom" instate of "bathroom". 


# https://subscription.packtpub.com/book/application-development/9781782167853/1/ch01lvl1sec16/calculating-wordnet-synset-similarity

In [ ]:
# define auxiliary functions

def getNouns(review):
  nouns = []
  cleanText = wordCleaner(review)
  tokens = word_tokenize(cleanText)
  taggedTokens = pos_tag(tokens, tagset='universal')
  for taggedToken in taggedTokens:
    (word, tokenType) = taggedToken
    if tokenType == "NOUN":
      nouns.append(word)
  return nouns

def wordCleaner(text):
  return re.sub("(\W(?!(\w)))+", " ", str(text)).lower()

def getSynsetsNames(synsets):
  return list(map(lambda synset: synset.name(), synsets))

rooms = getSynsetsNames(room.hyponyms()) # get room names as "constant", so we don't recalculate every time

In [ ]:
def findAllRoomMentions(hotels):
  counter = {}
  for index, hotel in tqdm.tqdm(hotels.iterrows()):
    reviews = getReviews(hotel)
    for reviewIdx, review in reviews.iterrows():
      counter = findRoomMentions(counter, review)
  return counter

def findRoomMentions(counter, review):
  text = str(review.title) + " " + str(review.review)
  tokenizedText = getNouns(text)
  for word in tokenizedText:
    try:
      roomType = getRoomType(word)
      if roomType != None:
        if roomType in counter:
          counter[roomType] += 1
        else:
          counter[roomType] = 1
    except Exception as e:
      print(e)
  return counter

def getRoomType(word):
  wordSynsets = wordnet.synsets(word)
  for wordSynset in getSynsetsNames(wordSynsets):
    if wordSynset in rooms:
      return wordSynset
  return None

# test
findAllRoomMentions(hotels[0:1])

In [ ]:
stats = {}

# for city in cities:
#   stats[city] = findAllRoomMentions(hotels[hotels["city"] == city])

print(stats)

### Resultados

## 2. ¿Qué servicios pueden detectarse por cada hotel?
Ejecutamos el mismo código de búsqueda anterior mortificándolo para que esta vez busque servicios.

In [ ]:
# buscar las definiciones de "que es un servicio"
hotelServices = ["restaurant", "pool", "spa", "gym", "bellman", "wifi", "television", "excursion", "clean"]
for service in hotelServices:
  for sense in wordnet.synsets(service):
      print(sense)
      print(sense.definition())
      print(sense.examples())
      print(sense.hyponyms())
      print(sense.hypernyms())
      print("-"*10)

In [ ]:
services = [
  wordnet.synset('service.n.15'), # el servicio que da un camarero a los clientes
  wordnet.synset('restaurant.n.01'),
  wordnet.synset('pool.n.01'),
  wordnet.synset('watering_place.n.01'), # kinda spa
  wordnet.synset('health_spa.n.01'),
  wordnet.synset('athletic_facility.n.01'),
  wordnet.synset('baggageman.n.01'),
  wordnet.synset('bellboy.n.01'),
  wordnet.synset('lifeguard.n.01'),
  wordnet.synset('checker.n.01'),
  wordnet.synset('houseclean.v.01'),
  wordnet.synset('wireless_local_area_network.n.01'),
  wordnet.synset('television.n.01'),
  wordnet.synset('excursion.n.01'),
]
serviceNames = getSynsetsNames(services)

In [ ]:
from collections import OrderedDict

def removeDuplicates(list):
  return OrderedDict.fromkeys(list).keys()

def getAllHypernyms(itemName):
  allItemHypernyms = []
  for synset in wordnet.synsets(itemName):
    allItemHypernyms = allItemHypernyms + getAllHypernymsOsSynset(synset)
  return removeDuplicates(allItemHypernyms)

def getAllHypernymsOsSynset(synset):
  synsetHypernyms = synset.hypernyms()
  if len(synsetHypernyms) == 0:
    return []

  allSynsetHypernyms = []
  for hypernym in synsetHypernyms:
    allSynsetHypernyms = allSynsetHypernyms + getAllHypernymsOsSynset(hypernym)

  return synsetHypernyms + allSynsetHypernyms

# test
print(getAllHypernyms('restaurant'))

In [ ]:
import pprint

def findAllServices(hotels):
  counterList = {}
  for index, hotel in tqdm.tqdm(hotels.iterrows()):
    reviews = getReviews(hotel)
    counter = {}
    for reviewIdx, review in reviews.iterrows():
      lookForServices(counter, review)
    counterList[hotel["hotel_name"]] = counter
  return counterList

def lookForServices(counter, review):
  text = str(review.title) + " " + str(review.review)
  tokenizedText = getNouns(text)
  for word in tokenizedText:
    try:
      serviceType = getServiceType(word)
      if serviceType != None:
        if serviceType in counter:
          counter[serviceType] += 1
        else:
          counter[serviceType] = 1
    except Exception as e:
      print(e)
  return counter

# En este caso en lugar de mirar si los synset de 1 nivel encajan con la lista de habitaciones posibles, vamos a hacerlo al revés.
# Vamos a coger todos los hypernyms de la palabra y ver si encaja 
def getServiceType(word):
  wordSynsets = getAllHypernyms(word)
  for wordSynset in getSynsetsNames(wordSynsets):
    if wordSynset in rooms:
      return wordSynset
  return None

# test
pprint.pprint(findAllServices(hotels[0:2]))

In [ ]:
pprint.pprint(findAllServices(hotels))